In [19]:
### Importing ###
from numpy import exp, array, random, dot #imports linear algebra operations/types
import numpy as np #operations in numpy
from scipy import misc #misc. operations in scipy
import skimage
import math 
from __future__ import with_statement

In [20]:
### Get picture data ###
arr1 = np.divide(misc.imread('save_save_save_america02.png'),255) #read 1st image, divide by 255 for number between 0 and 1
arr1r = arr1[:,:,0] #reds
arr1g = arr1[:,:,1] #greens
arr1b = arr1[:,:,2] #blues
arr2 = np.divide(misc.imread('tAmerica2.png'),255)
arr2r = arr2[:,:,0]
arr2g = arr2[:,:,1]
arr2b = arr2[:,:,2]

In [21]:
### Hyper-parameters ###
layerSize = 3 #number of filters in a layer
layerNum = 6 #number of layers
filterSize = 10 #size of filter
epochs = 10 #number of times trained
filters = [] #init filter
bias = 0.01
targets = np.ones((len(arr1r)*len(arr1r[0])*layerSize)) #target values

In [22]:
### Variables based on hyper-params ###
filterNum = layerSize**2 #number of filters per layer
totalFilterNum = filterNum*layerNum #total number of filters

In [23]:
### Set filters ###
def setFilters():
    global filters #Filters is a global variable
    filters = 0.01*random.random((layerNum, filterNum, filterSize, filterSize))-0.005 #Set filters to random 4d array
setFilters()
print(filters)

[[[[ -2.97503212e-03  -1.18400263e-03  -3.19878318e-03 ...,
     -1.71962154e-03   3.09708428e-03   1.32055781e-03]
   [  2.27452242e-03   4.64845135e-03   3.13510377e-03 ...,
      2.75874157e-04  -4.95095900e-04  -3.86889524e-03]
   [  2.82698253e-04   3.14475939e-03   4.48648830e-03 ...,
      7.85265919e-04   1.77927698e-03  -1.51387361e-03]
   ..., 
   [  1.12089997e-03   7.56549248e-04   1.67277674e-03 ...,
     -2.75174012e-03   1.61309563e-03  -5.29587530e-06]
   [  4.35643538e-03   9.67834295e-04   2.71358831e-03 ...,
      2.92106923e-03  -2.82116478e-03   1.28804263e-03]
   [  2.45295122e-03   2.89851774e-03  -2.10086756e-03 ...,
      1.15197053e-03  -4.74037900e-03   4.85134609e-03]]

  [[ -8.61885513e-06  -8.89151904e-04   4.52798783e-03 ...,
     -1.33630974e-03  -1.03885854e-03  -1.32199001e-03]
   [ -1.23291214e-03  -1.13140121e-04  -3.62185188e-03 ...,
     -1.89388627e-03  -8.90335599e-04   2.82973375e-03]
   [  4.06798720e-03   1.06378823e-03  -1.18328578e-03 ...,
 

In [98]:
### Helper functions ###
#Activation function (Leaky ReLU)
def activation(x, deriv=False, leak=-0.01):
    new_x=np.copy(array(x)) #Create a copy of x
    if deriv: #If you want to activate derivative
        #Calculation of derivative of leaky ReLU
        new_x[x > 0] = 1 
        new_x[x <= 0] = leak
        return new_x
    #Calculation of leaky ReLU
    new_x[x < 0] *= leak
    new_x = x * (x > 0)
    return new_x
#Sum with no negatives (not used)
'''
def sumNoNeg(arr):
    result = 0
    for y in range(len(arr)):
        for x in range(len(arr[0])):
            result += max(arr[y][x],0)
    return result
'''
#Apply cross-correlation for 1 index
def apply(filterArr, img, row, col):
    value = 0 #Init value
    halfFil = int(filterSize/2) #Half of filterSize
    padImg = np.pad(img, [(halfFil,halfFil),(halfFil,halfFil)],'constant', constant_values=(0)) #Pad image with 0's
    for x in range(filterSize): 
        for y in range(filterSize): #Apply over all filter
            value += padImg[x+row][y+col]*filterArr[x][y] #Add image multiplied by filter
    return value
#Softmax function
def softmax(x):
#     exps = np.exp(x - x.max())
#     return exps / np.sum(exps)
    return np.exp(x) / np.sum(np.exp(x), axis=0) #Calculate softmax
print(apply(filters[1][1],arr1r,0,0))

-138.248043855


In [30]:
### Apply filters ###
def applyFilters(img, new_img, filterArr):
    for y in range(len(img)):
        for x in range(len(img[y])): #Apply over all image
            new_img[y][x] += activation(apply(filterArr,img,y,x))+bias#Add up all cross-correlations (with ReLU)
    return new_img

In [31]:
### Pooling ###
def pool(img, size):
    pad_w = size-len(img)%size
    pad_h = size-len(img[0])%size
    if pad_w == size:
        pad_w = 0
    if pad_h == size:
        pad_h = 0
    new_img = np.lib.pad(array(img), ((0,pad_w),(0,pad_h)), 'edge') #Pad image
    k = len(new_img)/size #Width of new image 
    l = len(new_img[0])/size #Height of new image
    return new_img.reshape(int(k),size,int(l),size).max(axis=(-1,-3)) #Arrange image into groups of size*size, then get max on axis

In [117]:
### Forward-propagation ###
def forward(img):
    outputLayers = [[img[:,:,0],img[:,:,1],img[:,:,2]]] #Get all the image 
    for num in range(layerNum):
        outputLayers.append([]) #Append a new layer
        for lay in range(layerSize):
            outputLayers[num+1].append(np.zeros((len(outputLayers[num][lay]), len(outputLayers[num][lay][0]))))
            # Add components of the layer
            outputLayers[num][lay] = pool(outputLayers[num][lay], 2)
            # Do pooling
        #lay1 is the layer index you are at and lay2 is the layer index you are going to
        for lay1 in range(layerSize):
            for lay2 in range(layerSize):
                newImage = outputLayers[num+1][lay2] #Get refrence to next layer
#                 newImage += bias
#                 print("S-O: ("+str(len(outputLayers))+", "+str(len(outputLayers[0]))+")")
#                 print("S-F: ("+str(len(filters))+", "+str(len(filters[0]))+")")
                applyFilters(outputLayers[num][lay1], newImage, filters[num][layerSize*lay1+lay2])
                #Apply corresponding filter
                if (filterNum*num + layerSize*lay1+lay2) % 20 == 0:
                    print(filterNum*num + layerSize*lay1+lay2)
#                 print(newImage)
    outputLayers.append(softmax(array(outputLayers[len(outputLayers)-1]).flatten())) #Add flattened softmaxed array
    return outputLayers
out = forward(arr1)
print(out)

S-O: (2, 3)
S-F: (6, 9)
0
S-O: (2, 3)
S-F: (6, 9)
S-O: (2, 3)
S-F: (6, 9)
S-O: (2, 3)
S-F: (6, 9)
S-O: (2, 3)
S-F: (6, 9)
S-O: (2, 3)
S-F: (6, 9)
S-O: (2, 3)
S-F: (6, 9)
S-O: (2, 3)
S-F: (6, 9)
S-O: (2, 3)
S-F: (6, 9)
S-O: (3, 3)
S-F: (6, 9)
S-O: (3, 3)
S-F: (6, 9)
S-O: (3, 3)
S-F: (6, 9)
S-O: (3, 3)
S-F: (6, 9)
S-O: (3, 3)
S-F: (6, 9)
S-O: (3, 3)
S-F: (6, 9)
S-O: (3, 3)
S-F: (6, 9)
S-O: (3, 3)
S-F: (6, 9)
S-O: (3, 3)
S-F: (6, 9)
S-O: (4, 3)
S-F: (6, 9)
S-O: (4, 3)
S-F: (6, 9)
S-O: (4, 3)
S-F: (6, 9)
20
S-O: (4, 3)
S-F: (6, 9)
S-O: (4, 3)
S-F: (6, 9)
S-O: (4, 3)
S-F: (6, 9)
S-O: (4, 3)
S-F: (6, 9)
S-O: (4, 3)
S-F: (6, 9)
S-O: (4, 3)
S-F: (6, 9)
S-O: (5, 3)
S-F: (6, 9)
S-O: (5, 3)
S-F: (6, 9)
S-O: (5, 3)
S-F: (6, 9)
S-O: (5, 3)
S-F: (6, 9)
S-O: (5, 3)
S-F: (6, 9)
S-O: (5, 3)
S-F: (6, 9)
S-O: (5, 3)
S-F: (6, 9)
S-O: (5, 3)
S-F: (6, 9)
S-O: (5, 3)
S-F: (6, 9)
S-O: (6, 3)
S-F: (6, 9)
S-O: (6, 3)
S-F: (6, 9)
S-O: (6, 3)
S-F: (6, 9)
S-O: (6, 3)
S-F: (6, 9)
S-O: (6, 3)
S-F: (6, 9)
40
S-O: (6,

FloatingPointError: overflow encountered in double_scalars

In [118]:
### Error + Back-propagation ###
error = -np.sum(np.multiply(np.log10(out[len(out)-1]),targets)) # negative sum of log(out) times target- cross-entropy
print(error)
def back(layer, grad):
    if layer==0:
        return
    print(layer)
    #keep array of all gradients/changes/deltas
    new_grad = np.zeros((filterNum,filterSize,filterSize))
    for x in range(filterNum):
        r = 0
#         print("Filter S:"+str(len(filters)))
#         print(layerNum)
        for row in filters[layer-1][x]:
            c=0
            for w in row:
                new_grad[x][r][c]=np.multiply(error,np.mean(np.multiply(np.multiply(
                    activation(array(out[layer]),deriv=True),
                    w),
                    grad[x][r][c])))
                print(np.multiply(error,np.mean(np.multiply(np.multiply(
                    activation(array(out[layer]),deriv=True),
                    w),
                    grad[x][r][c]))))
#                 print(-np.divide(error,np.sum(np.multiply(np.multiply(out[layer-1], w), grad[x][r][c]))))
#                 print(new_grad[x][r][c])
                filters[layer-1][x][r][c]+=new_grad[x][r][c]
                c+=1
            r+=1
    back(layer-1, new_grad)
backout = back(layerNum, np.ones((filterNum,filterSize,filterSize)))
#take all output from prev. layer, multiply by corresponding gradient = new deltas 

3695.06365958
6
-2952520806.79
-813372140.292
1989441342.35
2401074044.73
2243039989.95
58516773.3386
637656400.89
-2208124785.28
731694299.194
3201176894.62
-2118438529.71
1314931048.36
1663660311.29
-3092764345.85
1722180131.75
1737126869.71
-1727673696.82
-635747971.422
-225011547.072
-2544828164.72
-3190406814.02
2671086850.29
3172750354.23
-1534569594.08
2737379553.13
917552562.92
-701525843.041
-3525488079.49
-1036887770.44
-1734412213.9
1849799017.75
-1360288092.09
2604776986.19
2184602358.23
-2927102999.09
1631896568.74
-2333034123.58
282577063.185
-1244378725.9
-379560228.061
-3541438089.41
3469732527.41
-1434098845.29
3437415464.79
-319818434.939
-3564170958.12
3013396158.24
2996483907.98
-990075092.928
-60363864.3154
-916131617.123
-1148591493.09
3407167776.25
376067587.895
-768330853.582
3107379424.73
173480454.425
1308311646.18
3383604145.74
673673207.237
-210079225.992
1736261116.43
-1939067895.97
3779972247.21
1446848787.37
-3144569188.23
-3112427568.22
-1892090048.71
37

FloatingPointError: overflow encountered in multiply

In [112]:
setFilters()
np.seterr(divide='raise')
np.seterr(invalid='raise')
np.seterr(over='raise')

{'divide': 'raise', 'invalid': 'raise', 'over': 'raise', 'under': 'ignore'}

In [116]:
for x in range(epochs):
    out = forward(arr1)
    error = -np.sum(np.multiply(np.log10(out[len(out)-1]),targets))
    print(error)
    print(filters)
    back(layerNum, np.ones((filterNum,filterSize,filterSize)))
#     print(filters)

0
20
40


FloatingPointError: overflow encountered in double_scalars